In [2]:
## Install the required packages
## pip install -qU elasticsearch openai
import os
from elasticsearch import Elasticsearch
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

es_client = Elasticsearch(
    hosts=os.environ["ELASTIC_SEARCH_URL"],
    api_key=os.environ["ES_API_KEY"]
)
      
openai_client = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
)
index_source_fields = {
    "ticker_watchlist": [
        "context_flattened"
    ]
}
def get_elasticsearch_results(query):
    es_query = {
        "retriever": {
            "rrf": {
                "retrievers": [
                    {
                        "standard": {
                            "query": {
                                "sparse_vector": {
                                    "field": "fundamental_summary_embedding",
                                    "inference_id": ".elser_model_2_linux-x86_64",
                                    "query": query
                                }
                            }
                        }
                    },
                    {
                        "standard": {
                            "query": {
                                "sparse_vector": {
                                    "field": "news_summary_embedding",
                                    "inference_id": ".elser_model_2_linux-x86_64",
                                    "query": query
                                }
                            }
                        }
                    },
                    {
                        "standard": {
                            "query": {
                                "sparse_vector": {
                                    "field": "technical_summary_embedding",
                                    "inference_id": ".elser_model_2_linux-x86_64",
                                    "query": query
                                }
                            }
                        }
                    }
                ]
            }
        }
    }
    result = es_client.search(index="ticker_watchlist", body=es_query)
    return result["hits"]["hits"]
def create_openai_prompt(question, results):
    context = ""
    for hit in results:
        inner_hit_path = f"{hit['_index']}.{index_source_fields.get(hit['_index'])[0]}"
        ## For semantic_text matches, we need to extract the text from the inner_hits
        if 'inner_hits' in hit and inner_hit_path in hit['inner_hits']:
            context += '\n --- \n'.join(inner_hit['_source']['text'] for inner_hit in hit['inner_hits'][inner_hit_path]['hits']['hits'])
        else:
            source_field = index_source_fields.get(hit["_index"])[0]
            hit_context = hit["_source"][source_field]
            context += f"{hit_context}\n"
    prompt = f"""
  Instructions:
  
  - You are an assistant for question-answering tasks. Please customize your responses for a stock picker analyst user.
  - Answer questions truthfully and factually using only the context presented.
  - If you don't know the answer, just say that you don't know, don't make up an answer.
  - You must always cite the document where the answer was extracted using inline academic citation style [], using the position.
  - Use markdown format for code examples.
  - You are correct, factual, precise, and reliable.
  
  Context:
  {context}
  
  """
    return prompt
def generate_openai_completion(user_prompt, question):
    response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": user_prompt},
            {"role": "user", "content": question},
        ]
    )
    return response.choices[0].message.content
if __name__ == "__main__":
    question = "What is the news summary for ALL?"
    elasticsearch_results = get_elasticsearch_results(question)
    context_prompt = create_openai_prompt(question, elasticsearch_results)
    openai_completion = generate_openai_completion(context_prompt, question)
    print(openai_completion)

For Allstate (ALL), the recent news summary indicates positive developments over the past three months. The company reported strong Q2 2024 earnings, exceeding analyst estimates with revenues of $15.82 billion and an EPS of $1.61. Following these announcements, the stock surged by 5%. Additionally, a notable $2 billion sale of its employer voluntary-benefits business aligns with Allstate's strategic focus on core insurance operations. Analysts have been optimistic, averaging a price target of $187.33. Overall, Allstate is regarded as a strong investment with high potential for growth due to improved premiums and strategic acquisitions, and recommendation sentiment is categorized as a "buy" [context].
